In [1]:
import sys
sys.path.append('methods')

from methods import Summarisation

In [2]:
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

In [3]:
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)

In [4]:
bench = Summarisation(URL=url, KEY=key, model_name='RefalMachine/RuadaptQwen3-32B-Instruct-v2', device=device, encoder=encoder) 

In [9]:
from utils import chunk_text
text = "\n".join(bench.collection[32]['text'])
chunks = chunk_text(text)
print(len(text))
print(len(chunks))

3927
1


In [8]:
print(bench.collection[32]['title'])

Канитель


In [10]:
s1 = await bench.blueprint.run(chunks, mode='default', initial_word_limit=500)

QUESTIONS
Создай план для следующего текста в виде списка ключевых вопросов, которые помогут понять основные элементы текста. 
Строго соблюдай правила:
- Вопросы должны выявлять главные события, персонажей, конфликты и важные детали.
- Не создавай более 15 вопросов, выбирая только самые существенные.
- Избегай повторяющихся по смыслу вопросов.
- Выводи только вопросы, каждый с новой строки, без номеров и дополнительных пояснений.

Текст:
---
КАНИТЕЛЬ На клиросе стоит дьячок Отлукавин и держит между вытянутыми жирными пальцами огрызенное гусиное перо. Маленький лоб его собрался в морщины, на носу играют пятна всех цветов, начиная от розового и кончая темно - синим. Перед ним на рыжем переплете Цветной триоди лежат две бумажки. На одной из них написано " о здравии ", на другой - " за упокой ", и под обоими заглавиями по ряду имен... Около клироса стоит маленькая старушонка с озабоченным лицом и с котомкой на спине. Она задумалась. - Дальше кого? - спрашивает дьячок, лениво почесывая за у

In [18]:
print(s1)

**Краткое содержание текста по плану:**

1. **Взаимосвязь характеров и локаций:** Мейзи связана с церковью и традициями жертвоприношений, миссис Уэст и полковник Уэст представляют современное отрицание наследия. Антикварий выступает хранителем фольклора, контрастирующего с рационализмом Уэстов. Замок Волверден-холл символизирует упадок аристократии, а церковь объединяет прошлое и настоящее через обряды, подчеркивающие социальные различия.

2. **Архитектура, история и природа в башне Волвердена:** Башня построена на основе языческих ритуалов, требовавших человеческих жертв для устойчивости. Природные катаклизмы разрушают её при нарушении баланса между людьми и стихиями, показав уязвимость цивилизации перед хаосом и магией.

3. **Церковь, легенда и современность:** Церковь возведена на месте древнего святилища, сохраняющем символы жертвоприношений и пророчеств. Ритуал с участием Мейзи реализует древнее пророчество, соединяя духовное наследие с современными трагедиями.

4. **Эмоциональное

In [6]:
import time
from utils import chunk_text

text = "\n".join(bench.collection[78]['text'])
chunks = chunk_text(text)
print(len(text))
print('chunks: ', len(chunks))

model_names = ['RefalMachine/RuadaptQwen3-32B-Instruct-v2', 'DeepSeek V3', 'tpro', 'yagpt5lite']
for name in model_names:
    print(name)
    bench.change_model(key, url, name)
    a1 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.blueprint.run(chunks, 500, 'default')
        end = time.perf_counter()
        c = end - start
        print(c)
        a1 += c
    res = a1 / 3
    print(f'Blueprint: {res:.2f}')
    a2 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.blueprint.run(chunks, 500, 'cluster')
        end = time.perf_counter()
        c = end - start
        print(c)
        a2 += c
    res = a2 / 3
    print(f'Blueprint cluster: {res:.2f}')
    a3 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.hierarchical.run(chunks, 500, False)
        end = time.perf_counter()
        c = end - start
        print(c)
        a3 += c
    res = a3 / 3
    print(f'Hierarchical: {res:.2f}')
    a4 = 0
    for _ in range(3):
        start = time.perf_counter()
        s = await bench.hierarchical.run(chunks, 500, True)
        end = time.perf_counter()
        c = end - start
        print(c)
        a4 += c
    res = a4 / 3
    print(f'Hierarchical filtered: {res:.2f}')
    print()
    break

yagpt5lite
99.0058389629703
101.04996362898964
99.03767797595356
Blueprint: 99.70
30.353947303025052
26.05761333007831
25.37439195497427
Blueprint cluster: 27.26
33.53323245793581
34.52723159804009
34.45615617907606
Hierarchical: 34.17
14.063398896949366
14.140152772073634
14.046449699089862
Hierarchical filtered: 14.08



In [6]:
lst = list(await bench.client.client.models.list())[0][1]
for l in lst:
    print(l)

Model(id='tpro', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-32B-Pro-Beta', created=None, object=None, owned_by=None, max_model_len=28000, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-7B-Garant-v2', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='deepseek-r1-32b', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='qwen32b-coder', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='vikhr12b', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='yagpt5lite', created=None, object=None, owned_by=None, max_model_len=30000, status='offloaded')
Model(id='qwen3moe30b', created=None, object=None, owned_by=None, 

In [1]:
import sys
sys.path.append('methods')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

In [2]:
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)

In [3]:
from utils import chunk_text
import asyncio
import json
from datetime import datetime, timezone
import aiofiles
import os
import time

async def run_bench_for_model(
    url: str,
    key: str,
    model_name: str,
    device = None,
    encoder = None,
    out_dir: str = "fixed_results_i_hope",
):

    file_path = f"{out_dir}/{model_name.replace('/', '_')}.jsonl"
    os.makedirs(out_dir, exist_ok=True)
    bench = Summarisation(URL=url, KEY=key, model_name=model_name, device=device, encoder=encoder)
    total_count = 0
    async with aiofiles.open(file_path, "a", encoding="utf‑8") as f:
        for item in bench.collection:
            if total_count == 100:
                print('DONE')
                break
            text = '\n'.join(item['text'])
            if len(text) > 800000:
                continue
            chunks = chunk_text(text)
            s1 = await bench.blueprint.run(chunks, initial_word_limit=500, mode='default')
            await f.write(_as_jsonl(model_name, item['title'], item['author'], "blueprint", s1))
            
            s2 = await bench.blueprint.run(chunks, initial_word_limit=500, mode='cluster')
            await f.write(_as_jsonl(model_name, item['title'], item['author'], "blueprint_cluster", s2))
            
            s3 = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=False)
            await f.write(_as_jsonl(model_name, item['title'], item['author'], "hierarchical", s3))
            
            s4 = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
            await f.write(_as_jsonl(model_name, item['title'], item['author'], "hierarchical_filtered", s4))
            
            total_count += 1

def _as_jsonl(model, book_title, author, method, annotation):
    entry = {
        "ts": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "model": model,
        "book_title": book_title,
        "author": author,
        "method": method,
        "annotation": annotation,
    }
    return json.dumps(entry, ensure_ascii=False) + "\n"

In [4]:
from utils import AsyncList

async def main():
    models = AsyncList()
    model_names = [
        'RefalMachine/RuadaptQwen3-32B-Instruct-v2'
        #'Qwen3-235B-A22B',
        #'DeepSeek V3',
        #'tpro'
        #'RefalMachine/RuadaptQwen2.5-7B-Lite-Beta',
        #'yagpt5lite'
    ]
    for name in model_names:
        models.append(run_bench_for_model(url, key, name, device, encoder))

    await models.complete_couroutines(batch_size=1)
    models = await models.to_list()

In [ ]:
await main()

In [3]:
bench = Summarisation(URL=url, KEY=key, model_name='yagpt5lite', device=device, encoder=encoder)

In [4]:
collection = bench.collection

In [5]:
import json

data = []
with open('results_for_true_model/RefalMachine_RuadaptQwen3-32B-Instruct-v2.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))

In [7]:
time_data = [e['ts'] for e in data]

In [8]:
from datetime import datetime, timedelta
import itertools
from collections import defaultdict

timestamps = time_data

datetimes = [datetime.fromisoformat(ts) for ts in timestamps]

category_cycle = itertools.cycle([1, 2, 3, 4])

category_seconds = defaultdict(list)

first_diff = timedelta(minutes=2)
first_cat = next(category_cycle)
category_seconds[first_cat].append(first_diff.total_seconds())

for prev, curr in zip(datetimes, datetimes[1:]):
    diff = curr - prev
    cat = next(category_cycle)
    if diff.total_seconds() > 1000:
        category_seconds[cat].append(180)
        continue
    category_seconds[cat].append(diff.total_seconds())

print("Среднее число секунд по категориям:")
for cat in sorted(category_seconds):
    diffs = category_seconds[cat]
    avg = sum(diffs) / len(diffs)
    print(f"Категория {cat}: {avg:.2f} секунд (на {len(diffs)} интервалах)")


Среднее число секунд по категориям:
Категория 1: 201.88 секунд (на 100 интервалах)
Категория 2: 140.41 секунд (на 100 интервалах)
Категория 3: 182.41 секунд (на 100 интервалах)
Категория 4: 147.32 секунд (на 100 интервалах)


In [6]:
tmp = {}
new_data = []
count = 0
for i in range(0, 400, 4):
    if i + 4 > len(data):
        break
    while data[i]['book_title'] != collection[count]['title']:
            #print(data[i]['book_title'])
            #print(collection[count]['title'])
        count += 1
    tmp = {
        'title': data[i]['book_title'], 
        'author': collection[count]['author'],
        'gold_annotation': collection[count]['annotation'],
        'annotations': {
            'blueprint': data[i]['annotation'],
            'blueprint_cluster': data[i + 1]['annotation'],
            'hierarchical': data[i + 2]['annotation'],
            'hierarchical_filtered': data[i + 3]['annotation']
        }
    }
    new_data.append(tmp)
    count += 1


In [7]:
print(len(new_data))

100


In [11]:
with open('RefalMachine_RuadaptQwen3-32B-Instruct-v2.json', 'w', encoding='utf-8') as file:
    json.dump(new_data, file, indent=4, ensure_ascii=False)

In [51]:
import json

with open('final/yandex_final (1).json', 'r', encoding='utf-8') as file:
    new_data = json.load(file)

In [8]:
r = []
b = []
for item in new_data:
    print(item['title'])
    tmp_r = []
    tmp_b = []
    for ann in item['annotations'].values():
        tmp_r.append(bench.evaluater.rouge_L(item['gold_annotation'], ann))
        tmp_b.append(bench.evaluater.bertscore(item['gold_annotation'], ann))
    r.append(tmp_r)
    b.append(tmp_b)

Башня Волверден
Алёша Птицын вырабатывает характер
Выбор
Апрельское колдовство
Будет ласковый дождь
И грянул гром
Вельд
Всё лето в один день
Земляничное окошко
Аугсбургский меловой круг
Все цвета радуги
День рождения Алисы
Девочка, с которой ничего не случится
Антоновские яблоки
Грамматика любви
Ида
Кастрюк
Кавказ
Князь во князьях
В деревне
Ворон
Золотое дно
Агафья
Альбом
Анюта
Белолобый
Беззащитное существо
Детвора
Драма на охоте
Горе
Гриша
Канитель
В аптеке
Враги
Злой мальчик
Злоумышленник
Короленко в кругу друзей
Гувернантка
Жгучая тайна
Жозеф Фуше
Избавилась
Возвращение
Бобок
Денискины рассказы
Белая дорога
Вакханки
Кактус
Дым в лесу
Голубая чашка
Горячий камень
Военная тайна
Как я мечтал о бескорыстии
Браун из Калавераса
Джентльмен из Лапорта
Король забавляется
Дети солнца
Егор Булычов и другие
В людях
Картина
Без улыбок
Искатель приключений
Жизнь Гнора
Королевство кривых зеркал
Деревянная пастушка
Карлик Нос
Гедда Габлер
Багульник
Бамбус
Баваклава
Верный друг Санчо
Всадник, скачу

In [9]:
methods = [
    "blueprint",
    "blueprint_cluster",
    "hierarchical",
    "hierarchical_filtered",
]

row = []
for r1, b1 in zip(r, b):
    for r2, b2, m in zip(r1, b1, methods):
        row.append({
            'model': 'RefalMachine_RuadaptQwen3-32B-Instruct-v2',
            'method': m,
            'metric': 'rouge-l',
            'score': r2
        })
        row.append({
            'model': 'RefalMachine_RuadaptQwen3-32B-Instruct-v2',
            'method': m,
            'metric': 'bertscore',
            'score': b2[2]
        })

In [10]:
import pandas as pd

df = pd.DataFrame(row)

In [11]:
agg = (
    df
    .groupby(["model", "metric", "method"])
    .agg(mean=("score", "mean"), std=("score", "std"))
    .round(3)
    .reset_index()
)

In [12]:
mean_tbl = agg.pivot(index=["model","metric"], columns="method", values="mean")
std_tbl  = agg.pivot(index=["model","metric"], columns="method", values="std")

In [13]:
final = mean_tbl.astype(str) + " ± " + std_tbl.astype(str)

In [14]:
final = final[methods]

# === 5. Готовим к LaTeX: multiindex → колонки + индекс ===
final_df = final.reset_index()
# экранируем подчёркивания в названиях
final_df.columns = final_df.columns.str.replace("_", r"\_", regex=False)

In [49]:
print(final)

method                                                  blueprint  \
model                                    metric                     
RefalMachine_RuadaptQwen2.5-7B-Garant-v2 bertscore  0.587 ± 0.038   
                                         rouge-l    0.141 ± 0.048   

method                                             blueprint_cluster  \
model                                    metric                        
RefalMachine_RuadaptQwen2.5-7B-Garant-v2 bertscore     0.577 ± 0.037   
                                         rouge-l       0.122 ± 0.045   

method                                               hierarchical  \
model                                    metric                     
RefalMachine_RuadaptQwen2.5-7B-Garant-v2 bertscore  0.596 ± 0.035   
                                         rouge-l    0.144 ± 0.043   

method                                             hierarchical_filtered  
model                                    metric                           
RefalMa

In [21]:
# превращаем в DataFrame с нормальными строками
final_df = final.reset_index().rename(columns={"metric": "Metric"})
# выводим в LaTeX
print(final_df.to_latex(index=False))


\begin{tabular}{lllll}
\toprule
Metric & blueprint & blueprint_cluster & hierarchical & hierarchical_filtered \\
\midrule
bertscore & 0.587 ± 0.038 & 0.577 ± 0.037 & 0.596 ± 0.035 & 0.593 ± 0.034 \\
rouge-l & 0.141 ± 0.048 & 0.122 ± 0.045 & 0.144 ± 0.043 & 0.138 ± 0.043 \\
\bottomrule
\end{tabular}



In [15]:
# превращаем в DataFrame с нормальными строками
final_df = final.reset_index().rename(columns={"metric": "Metric"})
# выводим в LaTeX
print(final_df.to_latex(index=False))

\begin{tabular}{llllll}
\toprule
model & Metric & blueprint & blueprint_cluster & hierarchical & hierarchical_filtered \\
\midrule
RefalMachine_RuadaptQwen3-32B-Instruct-v2 & bertscore & 0.569 ± 0.058 & 0.539 ± 0.052 & 0.557 ± 0.034 & 0.557 ± 0.036 \\
RefalMachine_RuadaptQwen3-32B-Instruct-v2 & rouge-l & 0.104 ± 0.045 & 0.076 ± 0.036 & 0.103 ± 0.029 & 0.1 ± 0.025 \\
\bottomrule
\end{tabular}



In [86]:
# превращаем в DataFrame с нормальными строками
final_df = final.reset_index().rename(columns={"metric": "Metric"})
# выводим в LaTeX
print(final_df.to_latex(index=False))

\begin{tabular}{llllll}
\toprule
model & Metric & blueprint & blueprint_cluster & hierarchical & hierarchical_filtered \\
\midrule
Qwen3-235B-A22B & bertscore & 0.616 ± 0.033 & 0.593 ± 0.034 & 0.612 ± 0.03 & 0.609 ± 0.027 \\
Qwen3-235B-A22B & rouge-l & 0.158 ± 0.045 & 0.122 ± 0.036 & 0.149 ± 0.04 & 0.148 ± 0.037 \\
\bottomrule
\end{tabular}



In [98]:
# превращаем в DataFrame с нормальными строками
final_df = final.reset_index().rename(columns={"metric": "Metric"})
# выводим в LaTeX
print(final_df.to_latex(index=False))

\begin{tabular}{llllll}
\toprule
model & Metric & blueprint & blueprint_cluster & hierarchical & hierarchical_filtered \\
\midrule
DeepSeek V3 & bertscore & 0.58 ± 0.04 & 0.584 ± 0.036 & 0.6 ± 0.031 & 0.6 ± 0.029 \\
DeepSeek V3 & rouge-l & 0.126 ± 0.046 & 0.112 ± 0.039 & 0.137 ± 0.039 & 0.135 ± 0.037 \\
\bottomrule
\end{tabular}



In [137]:
# превращаем в DataFrame с нормальными строками
final_df = final.reset_index().rename(columns={"metric": "Metric"})
# выводим в LaTeX
print(final_df.to_latex(index=False))

\begin{tabular}{llllll}
\toprule
model & Metric & blueprint & blueprint_cluster & hierarchical & hierarchical_filtered \\
\midrule
tpro & bertscore & 0.59 ± 0.049 & 0.582 ± 0.037 & 0.594 ± 0.03 & 0.595 ± 0.033 \\
tpro & rouge-l & 0.147 ± 0.049 & 0.118 ± 0.039 & 0.138 ± 0.031 & 0.135 ± 0.03 \\
\bottomrule
\end{tabular}



In [63]:
latex = final_df.to_latex(
    index=False,
    escape=False,
    column_format="l" + "c"*len(methods),
    multicolumn=True,
    multirow=True,      # pandas ≥2.1 добавит \multirow для повторяющихся model
    caption="Результаты по методам и моделям",
    label="tab:results_models",
)

print(latex)

\begin{table}
\caption{Результаты по методам и моделям}
\label{tab:results_models}
\begin{tabular}{lcccc}
\toprule
model & metric & blueprint & blueprint\_cluster & hierarchical & hierarchical\_filtered \\
\midrule
yagpt5lite & bertscore & 0.61 ± 0.036 & 0.614 ± 0.032 & 0.623 ± 0.032 & 0.621 ± 0.033 \\
yagpt5lite & rouge-l & 0.158 ± 0.052 & 0.14 ± 0.044 & 0.167 ± 0.05 & 0.165 ± 0.047 \\
\bottomrule
\end{tabular}
\end{table}



In [23]:
print(final)

method         blueprint blueprint_cluster   hierarchical  \
metric                                                      
bertscore  0.587 ± 0.038     0.577 ± 0.037  0.596 ± 0.035   
rouge-l    0.141 ± 0.048     0.122 ± 0.045  0.144 ± 0.043   

method    hierarchical_filtered  
metric                           
bertscore         0.593 ± 0.034  
rouge-l           0.138 ± 0.043  
